In [1]:
import argparse
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import scipy.io
import scipy.misc
import numpy as np
import pandas as pd
import PIL
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Lambda, Conv2D
from keras.models import load_model, Model
import h5py
from PIL import Image

from yolo_utils import read_classes, read_anchors, generate_colors, preprocess_image, draw_boxes, scale_boxes
from keras_yolo import yolo_head, yolo_boxes_to_corners, preprocess_true_boxes, yolo_loss, yolo_body

#Functions ****************

def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = .6):

    box_scores = tf.multiply(box_confidence,box_class_probs)

    box_classes = K.argmax(box_scores, axis=-1)
    box_class_scores = K.max(box_scores, axis=-1)    

    filtering_mask = tf.greater(box_class_scores,threshold)
    
    scores = tf.boolean_mask(box_class_scores,filtering_mask)
    boxes = tf.boolean_mask(boxes,filtering_mask)
    classes = tf.boolean_mask(box_classes,filtering_mask)
    
    return scores, boxes, classes


def iou(box1, box2):

    xi1 = np.maximum(box1[0],box2[0])
    yi1 = np.maximum(box1[1],box2[1])
    xi2 = np.minimum(box1[2],box2[2])
    yi2 = np.minimum(box1[3],box2[3])
    inter_area = max(yi2 - yi1, 0)*max(xi2 - xi1, 0)

    box1_area = (box1[3] - box1[1])*(box1[2]-box1[0])
    box2_area = (box2[3] - box2[1])*(box2[2]-box2[0])
    union_area = box1_area + box2_area - inter_area
    
    iou = inter_area/ union_area

    return iou


def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
    
    max_boxes_tensor = K.variable(max_boxes, dtype='int32')
    K.get_session().run(tf.variables_initializer([max_boxes_tensor]))
    
    nms_indices = tf.image.non_max_suppression(boxes,scores,max_boxes_tensor,iou_threshold)
    
    scores = K.gather(scores,nms_indices)
    boxes = K.gather(boxes,nms_indices)
    classes = K.gather(classes,nms_indices)
    
    return scores, boxes, classes


def yolo_eval(yolo_outputs, image_shape = (720., 1280.), max_boxes=10, score_threshold=.6, iou_threshold=.5):
    
    box_confidence, box_xy, box_wh, box_class_probs = yolo_outputs
    boxes = yolo_boxes_to_corners(box_xy, box_wh)
    scores, boxes, classes = yolo_filter_boxes(box_confidence,boxes,box_class_probs,score_threshold) 
    boxes = scale_boxes(boxes, image_shape)
    scores, boxes, classes = yolo_non_max_suppression(scores,boxes,classes,max_boxes,iou_threshold)
    
    return scores, boxes, classes


def predict(sess, image_data):
   
    image, image_data = preprocess_image(image_file, model_image_size = (608, 608))
    
    out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes],{yolo_model.input:image_data,K.learning_phase(): 0})

    print('Found {} boxes for {}'.format(len(out_boxes), image_data))
    colors = generate_colors(class_names)
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    image.save(os.path.join(r"C:\Users\User\Desktop\Project2", image_file), quality=90)
    output_image = scipy.misc.imread(os.path.join(r"C:\Users\User\Desktop\Project2", image_file))
    imshow(output_image)
    
    return out_scores, out_boxes, out_classes

Using TensorFlow backend.


In [2]:
class_names = read_classes("coco_classes.txt")
anchors = read_anchors("yolo_anchors.txt")
image_shape = (720., 1280.) 

In [3]:
sess = K.get_session()

In [4]:
yolo_model = load_model("yolo.h5")

Instructions for updating:
Colocations handled automatically by placer.


C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [5]:
yolo_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 608, 608, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 608, 608, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 608, 608, 32) 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, 608, 608, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [8]:
yolo_outputs = yolo_head(yolo_model.output, anchors, len(class_names))
scores, boxes, classes = yolo_eval(yolo_outputs, image_shape)

In [12]:
out_scores, out_boxes, out_classes = predict(sess, "0055.jpg")

NameError: name 'image_file' is not defined

In [13]:
import cv2

cap = cv2.VideoCapture(0)

while(True):
    
    ret, frame = cap.read()
    image = Image.fromarray(frame, 'RGB')
    image_data = cv2.resize(frame, (608,608))
    #image_data /= 255.
    image_data = np.expand_dims(image_data, 0)
    out_scores, out_boxes, out_classes = sess.run([scores, boxes, classes],{yolo_model.input:image_data,K.learning_phase(): 0})
    print('Found {} boxes'.format(len(out_boxes)))
    colors = generate_colors(class_names)   
    draw_boxes(image, out_scores, out_boxes, out_classes, class_names, colors)
    
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


Found 0 boxes


TypeError: 'int' object is not subscriptable